In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd drive/My Drive/P1_Facial_Keypoints

/content/drive/My Drive/P1_Facial_Keypoints


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import os
import cv2

In [ ]:
test_df=pd.read_csv("data/test_frames_keypoints.csv")
train_df=pd.read_csv("data/training_frames_keypoints.csv")

In [ ]:
test_df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135
0,James_Wolfensohn_00.jpg,25.0,44.0,25.0,49.0,25.0,54.0,25.0,59.0,26.0,64.0,27.0,67.0,30.0,71.0,33.0,75.0,39.0,76.0,46.0,76.0,52.0,74.0,56.0,72.0,59.0,67.0,62.0,62.0,63.0,57.0,64.0,52.0,65.0,47.0,27.0,40.0,30.0,39.0,32.0,...,33.0,65.0,36.0,64.0,38.0,64.0,39.0,65.0,41.0,64.0,44.0,65.0,47.0,65.0,44.0,66.0,42.0,68.0,39.0,68.0,37.0,68.0,36.0,66.0,33.0,65.0,38.0,65.0,39.0,65.0,42.0,65.0,46.0,65.0,42.0,65.0,39.0,65.0,38.0,65.0
1,Valerie_Harper_30.jpg,55.0,62.0,52.0,72.0,52.0,83.0,55.0,90.0,55.0,101.0,57.0,108.0,60.0,114.0,65.0,121.0,75.0,127.0,88.0,124.0,98.0,122.0,106.0,117.0,114.0,112.0,122.0,102.0,124.0,94.0,129.0,87.0,129.0,76.0,57.0,52.0,60.0,50.0,65.0,...,65.0,101.0,68.0,101.0,70.0,98.0,75.0,101.0,78.0,101.0,83.0,104.0,88.0,106.0,83.0,109.0,78.0,109.0,73.0,109.0,70.0,109.0,68.0,106.0,65.0,101.0,70.0,104.0,75.0,104.0,78.0,104.0,88.0,106.0,78.0,106.0,75.0,104.0,70.0,104.0
2,Kristen_Breitweiser_11.jpg,60.0,88.0,60.0,98.0,63.0,109.0,65.0,116.0,68.0,127.0,73.0,135.0,78.0,137.0,83.0,140.0,96.0,143.0,109.0,138.0,119.0,135.0,127.0,130.0,132.0,125.0,135.0,115.0,135.0,105.0,137.0,95.0,137.0,84.0,63.0,75.0,65.0,70.0,71.0,...,78.0,119.0,83.0,114.0,89.0,112.0,91.0,112.0,94.0,112.0,101.0,112.0,109.0,117.0,101.0,120.0,96.0,122.0,94.0,122.0,89.0,122.0,83.0,122.0,81.0,119.0,89.0,117.0,91.0,114.0,96.0,114.0,106.0,117.0,96.0,117.0,94.0,117.0,89.0,117.0
3,Elizabeth_Dole_31.jpg,41.0,96.0,43.0,104.0,47.0,113.0,49.0,120.0,56.0,128.0,64.0,135.0,73.0,139.0,83.0,144.0,96.0,144.0,105.0,137.0,107.0,133.0,107.0,127.0,109.0,120.0,109.0,112.0,109.0,103.0,109.0,95.0,107.0,86.0,60.0,83.0,66.0,79.0,73.0,...,83.0,122.0,90.0,120.0,94.0,116.0,98.0,116.0,100.0,116.0,103.0,116.0,103.0,118.0,103.0,122.0,100.0,125.0,98.0,127.0,94.0,127.0,90.0,124.0,83.0,122.0,94.0,120.0,96.0,118.0,98.0,118.0,103.0,118.0,98.0,120.0,96.0,122.0,94.0,122.0
4,Kit_Bond_20.jpg,35.0,67.0,37.0,74.0,41.0,81.0,44.0,87.0,48.0,96.0,51.0,103.0,58.0,108.0,66.0,112.0,76.0,112.0,85.0,107.0,91.0,100.0,94.0,93.0,96.0,84.0,96.0,77.0,96.0,70.0,96.0,64.0,94.0,55.0,42.0,67.0,48.0,65.0,51.0,...,60.0,94.0,64.0,96.0,69.0,94.0,73.0,94.0,74.0,94.0,80.0,94.0,83.0,93.0,80.0,98.0,78.0,101.0,73.0,101.0,69.0,101.0,66.0,99.0,60.0,94.0,69.0,96.0,73.0,96.0,76.0,96.0,83.0,93.0,76.0,98.0,73.0,98.0,69.0,98.0


In [ ]:

import torch
from models import resnet18_grayscale

img_size = 224

#
resnet18 = resnet18_grayscale()

## load the best saved model parameters (by your path name)
## You'll need to un-comment the line below and add the correct name for *your* saved model
model_dir = 'saved_models/'
model_name = 'resnet18_gray_epochs46_bs64_vloss0.00524.pt'

resnet18.load_state_dict(torch.load(model_dir+model_name))

resnet18.eval()

resnet18_grayscale(
  (resnet18): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tru

In [ ]:
n = 2
image_name =test_df.iloc[n, 0]
image = mpimg.imread(os.path.join('data/test/', image_name))

In [ ]:
image= cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
image=image/255
h,w=image.shape

In [ ]:
h,w

(191, 184)

In [ ]:
image= cv2.resize(image, (img_size, img_size))

In [ ]:
image.shape

(224, 224)

In [ ]:
im_copy=np.copy(image)

In [ ]:
# if image has no grayscale color channel, add one
if(len(image.shape) == 2):
            # add that third color dim
  image= image.reshape(image.shape[0],image.shape[1], 1)

In [ ]:
image.shape

(224, 224, 1)

In [ ]:
# swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
image=image.transpose((2, 0, 1))
        
image= torch.from_numpy(image)
image= image.type(torch.FloatTensor)
print(image.shape)        
image=image.unsqueeze_(0)

torch.Size([1, 224, 224])


In [ ]:
image.shape

torch.Size([1, 1, 224, 224])

In [ ]:
im_copy.shape

(224, 224)

In [ ]:
# forward pass to get net output
output_pts =resnet18(image)
print(output_pts.shape)
# reshape to size x 68 x 2 pts
output_pts = output_pts.view(68, -1)
        
        # undo normalization of keypoints
output_pts = output_pts.detach().numpy()   
output_pts = output_pts*(im_copy.shape[0]/4)+im_copy.shape[0]/2

torch.Size([1, 136])


In [ ]:
output_pts.shape

(68, 2)

In [ ]:
output_pts=output_pts*[h/224,w/224]

In [ ]:
def overall_r2_score(df=test_df,image_path="data/test"):
  mse=[]
  r2=[]
  for i in range(len(df)):
    image_name=df.iloc[i,0]
    image=mpimg.imread(os.path.join(image_path,image_name))
    image=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    image=image/255
    h,w=image.shape

    image=cv2.resize(image,(img_size,img_size))
    
    if len(image.shape)==2:
      image=image.reshape(image.shape[0],image.shape[1],1)

    image=image.transpose((2, 0, 1))
    image= torch.from_numpy(image)
    image= image.type(torch.FloatTensor)
    image=image.unsqueeze_(0)
    # forward pass to get net output
    output_pts =resnet18(image)
   
    # reshape to size x 68 x 2 pts
    output_pts = output_pts.view(68, -1)
        
        # undo normalization of keypoints
    output_pts = output_pts.detach().numpy()   
    output_pts = output_pts*(im_copy.shape[0]/4)+im_copy.shape[0]/2

    #undo rescaling
    output_pts=output_pts*[h/img_size,w/img_size]

    output_pts=output_pts.flatten()
    actual_y=df.iloc[i,1:].values

    mse.append(mean_squared_error(actual_y,output_pts))
    r2.append(r2_score(actual_y,output_pts))
  return mse,r2

In [ ]:
mse,r2=overall_r2_score(train_df,'data/training')

In [ ]:
r2

In [ ]:
mse

In [ ]:
output_pts=output_pts.reshape(136,-1)

In [ ]:
output_pts[:10:2].flatten()

array([66.36910105, 66.81439713, 69.09764385, 71.31445721, 74.51704356])

In [ ]:
test_df.iloc[2,1:11:2].values

array([60.0, 60.0, 63.0, 65.0, 68.0], dtype=object)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
np.sqrt(mean_squared_error(test_df.iloc[2,1:].values,output_pts.flatten()))

4.074159498304367

In [ ]:
r2_score(test_df.iloc[2,1:].values,output_pts.flatten())

0.9627886980310508

In [ ]:
r2_score([4,7,3],[4.2,6.7,2.5])

0.9561538461538461

In [ ]:
r2_score([5,6,7],[4.3,6.2,7.5])

0.6099999999999999

In [ ]:
r2_score([4,7,3,5,6,7],[4.2,6.7,2.5,4.3,6.2,7.5])

0.9129999999999999